In [304]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show 
import matplotlib.patches as patches    
from matplotlib import colors


import glob

import subprocess
from subprocess import call

import re

import pickle

#from Bio.SubsMat import MatrixInfo
#import Bio.Data.CodonTable

from scipy import stats


from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

#import joypy

import pyBigWig

* Peptide: Sequence of the peptide
* Gene_Names: The Genes that this peptide maps to. Per your suggestion, I've gotten rid of any peptides that matched to more than one gene. Also note that many peptides do not come from an ENSEMBL annotated gene and are thus left as '-'. 
* BestPSM_PEP: This is the Posterior Error Probability of the peptide. If you wanted to rank the peptides by some MS score, this is the best. 
* ENST_ids: All of the ENST ids that this peptide could come from. Notice that many of the peptide will come from the same gene but we are unable to distinguish between different transcripts. 
* Protein_Accessions: The accessions of the proteins it matched to. This is probably the least helpful column for you
* Chromosome/scaffold name: Name of chromosome
* Gene start (bp): Where we can map the peptide of the genome, this is where the gene starts 
* Gene end (bp): Where the gene ends
* Parsed_Loci: For some peptides that do not map to ENSEMBL the original curators of the database have added genomic co-ordianates (which you can see in the protein_accessions column). This column just parses those locations. 


In [345]:
df = pd.read_csv('DATA/Novels_forAlla.txt', sep='\t')
df

,Peptide,Gene_Names,BestPSM_PEP,ENST_ids,Protein_Accessions,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Parsed_Loci
0,AAAAPESF,CIZ1,0.010166,ENST00000476727,NonCoding_2_ENST00000476727_ENSG00000148337,9,128161251,128204383,-
1,AADGHHRGL,HSPBP1,0.001552,"ENST00000255631,ENST00000433386,ENST0000058569...","10_ENST00000255631_ENSG00000133265_F1_orf2,10_...",19,55262223,55280381,-
2,AADHGPDEY,POMK,0.001915,"ENST00000331373,ENST00000518991,ENST0000061442...","19_ENST00000331373_ENSG00000185900_F2_orf2,19_...",8,43093498,43131180,-
3,AAEPVDSSL,-,0.046727,-,ERV_1792085_ERVL-E-int::chr18:74213725-74215142_6,-,-,-,chr18:74213725-74215142
4,AAFAAPVKA,DPY19L2,0.021480,ENST00000538147,"2_ENST00000538147_ENSG00000177990_F0_orf2,UTR5...",12,63558913,63668939,-
...,...,...,...,...,...,...,...,...,...
1518,YTADHGHVY,CLDND1,0.000701,ENST00000503004,28_ENST00000503004_ENSG00000080822_F1_orf11,3,98497912,98523066,-
1519,YTDHAHPLHRGDS,-,0.040256,-,ERV_1270162_MLT1K_MLT1K-int::chr15:29477473-29...,-,-,-,chr15:29477473-29480097
1520,YTDHSWLDQRW,-,0.024654,-,ERV_2415898_MLT1M::chr21:37752264-37752370_1,-,-,-,chr21:37752264-37752370
1521,YTLVHAGP,PSMF1,0.026704,ENST00000335877,"153_ENST00000335877_ENSG00000125818_F2_orf81,U...",20,1113240,1189415,-


In [306]:
metadata_pc_g25 =  pd.read_csv('/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/tmp_res/metadata_pc_g25.txt', sep='\t')
metadata_pc_g25['tr_id1'] = [x.split('.')[0] for x in metadata_pc_g25['tr_id'].tolist()]
metadata_pc_g25[0:3]

,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id,tr_id1
0,ENST00000335137.3,OR4F5,OR4F5-001,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,NaN,ATG,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,ATG,TAG,0,918,0,0,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,ENST00000335137
1,ENST00000624431.1,FO538757.2,FO538757.2-201,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,AAG,ATGAGTGACAGCATCAACTTCTCTCACAACCTAGGCCAGCTCCTGT...,ATG,TAG,316,718,0,316,ENST00000624431.1|ENSG00000279928.1|-|-|FO5387...,ENST00000624431
2,ENST00000623834.3,FO538757.1,FO538757.1-201,ATGACTCCTGTGAGGATGCAGCACTCCCTGGCAGGTCAGACCTATG...,NaN,ATG,ATGACTCCTGTGAGGATGCAGCACTCCCTGGCAGGTCAGACCTATG...,ATG,TAG,0,1080,0,0,ENST00000623834.3|ENSG00000279457.3|-|-|FO5387...,ENST00000623834


In [307]:
gtf = pd.read_csv('DATA/gencode.v25.annotation.gtf', sep='\t', comment='#', header=None)
gtf_tr = gtf[gtf[2] == 'transcript']

In [308]:
gtf_tr['tr_id'] = [x.split('transcript_id ')[1].split('"')[1] for x in gtf_tr[8].tolist()]
gtf_tr['tr_id1'] = [x.split('.')[0] for x in gtf_tr['tr_id'].tolist()]

gtf_tr[0:2]

/tmp/ipykernel_74992/2591424268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gtf_tr['tr_id'] = [x.split('transcript_id ')[1].split('"')[1] for x in gtf_tr[8].tolist()]
/tmp/ipykernel_74992/2591424268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gtf_tr['tr_id1'] = [x.split('.')[0] for x in gtf_tr['tr_id'].tolist()]


,0,1,2,3,4,5,6,7,8,tr_id,tr_id1
1,chr1,HAVANA,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENST00000456328.2,ENST00000456328
5,chr1,HAVANA,transcript,12010,13670,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENST00000450305.2,ENST00000450305


In [274]:
gtf_tr.iloc[0][8]

'gene_id "ENSG00000223972.5"; transcript_id "ENST00000456328.2"; gene_type "transcribed_unprocessed_pseudogene"; gene_status "KNOWN"; gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_status "KNOWN"; transcript_name "DDX11L1-002"; level 2; transcript_support_level "1"; tag "basic"; havana_gene "OTTHUMG00000000961.2"; havana_transcript "OTTHUMT00000362751.1";'

# We need to select reasonable rank for each category to make a cutoff 

In [506]:
# rank 5000
uORF_df = pd.read_csv('DATA/NonCanonicalORFs_Trips-viz_10studies_100samples_1000/AUG_uORF_1000_10studies.csv')

# rank y
uoORF_df = pd.read_csv('DATA/NonCanonicalORFs_Trips-viz_10studies_100samples_1000/AUG_uoORF_1000_10studies.csv')

# rank x
doORF_df = pd.read_csv('DATA/NonCanonicalORFs_Trips-viz_10studies_100samples_1000/AUG_doORF_1000_10studies.csv')

# rank u
dORF_df = pd.read_csv('DATA/NonCanonicalORFs_Trips-viz_10studies_100samples_1000/AUG_dORF_1000_10studies.csv')

# rank a
lncORF_df = pd.read_csv('DATA/NonCanonicalORFs_Trips-viz_10studies_100samples_1000/AUG_lncORF_1000_10studies.csv')


In [507]:
# rank 
nORF_df = pd.read_csv('DATA/NonCanonicalORFs_Trips-viz_10studies_100samples_1000/AUG_nORF_1000_10studies.csv').drop_duplicates()
nORF_df.columns = ['Gene', 'tr_id1', 'Start', 'Stop', 'Length', 'Global_Rank', 'Type',
       'Trips-viz link', 'Start Codon', 'Highframe rank', 'Highframe value',
       'Lowframe rank', 'Lowframe value', 'Stop rank', 'Stop value',
       'Start rank', 'Start value', 'Coverage rank', 'Coverage value',
       'Inframe Count Rank', 'Inframe Count Value', 'Amino acid sequence',
       'Proteomics count', 'Read density']

# filter reading frame of nORFs 
nORF_df[0:2]

nORF_df1 = nORF_df.merge(metadata_pc_g25[['tr_id1', 'cds_start_pos']],
                        on='tr_id1', how='inner').drop_duplicates()


li = []
for el in nORF_df1[['tr_id1', 'Start', 'cds_start_pos']].to_numpy():
    tr_id1 = el[0]
    start = el[1]-1
    cds_start = el[2]
    
    if (cds_start % 3) == (start % 3):
        # same frame 
        frame = 'the same'
    else:
        frame = 'different'
        
    li.append([tr_id1, el[1], frame])
    
frame_df = pd.DataFrame(li, columns = ['tr_id1', 'Start', 'frame'])
    
nORF_df2 = nORF_df1.merge(frame_df,
                         on=['tr_id1', 'Start'], how='inner').drop_duplicates()


# if frame is the same it's probably lncRNA or wrong CDS annotation with no 5'UTR
nORF_df3 = nORF_df2[nORF_df2['frame'] == 'different']

print (nORF_df2.shape[0], nORF_df1.shape[0], nORF_df.shape[0], nORF_df3.shape[0])

# pseudogenes
# set(nORF_df.tr_id1.tolist()) - set(nORF_df2.tr_id1.tolist())

145334 145334 145448 145240


# Combine all ORFs together using rank 

In [495]:
uORF_sel = uORF_df[['Gene', 'Tran', 'Start', 'Stop', 'Global_Rank', 'Type', 
         'Amino acid sequence']][0:6200]

uORF_sel.columns = ['Gene', 'tr_id1', 'Start', 
                    'Stop', 'Global_Rank_within_ORF_type', 
                    'Type_of_ORF', 'Amino acid sequence']

In [496]:
nORF_sel = nORF_df3[['Gene', 'tr_id1', 'Start', 'Stop', 'Global_Rank', 'Type', 
         'Amino acid sequence']].sort_values(by='Global_Rank')[0:945]

nORF_sel.columns = ['Gene', 'tr_id1', 'Start', 
                    'Stop', 'Global_Rank_within_ORF_type', 
                    'Type_of_ORF', 'Amino acid sequence']

In [497]:
uoORF_sel = uoORF_df[['Gene', 'Tran', 'Start', 'Stop', 'Global_Rank', 'Type', 
         'Amino acid sequence']][0:800]

uoORF_sel.columns = ['Gene', 'tr_id1', 'Start', 
                    'Stop', 'Global_Rank_within_ORF_type', 
                    'Type_of_ORF', 'Amino acid sequence']

In [498]:
dORF_sel = dORF_df[['Gene', 'Tran', 'Start', 'Stop', 'Global_Rank', 'Type', 
         'Amino acid sequence']][0:350]

dORF_sel.columns = ['Gene', 'tr_id1', 'Start', 
                    'Stop', 'Global_Rank_within_ORF_type', 
                    'Type_of_ORF', 'Amino acid sequence']

In [499]:
doORF_sel = doORF_df[['Gene', 'Tran', 'Start', 'Stop', 'Global_Rank', 'Type', 
         'Amino acid sequence']][0:110]

doORF_sel.columns = ['Gene', 'tr_id1', 'Start', 
                    'Stop', 'Global_Rank_within_ORF_type', 
                    'Type_of_ORF', 'Amino acid sequence']

In [500]:
lncORF_sel = lncORF_df[['Gene', 'Tran', 'Start', 'Stop', 'Global_Rank', 'Type', 
         'Amino acid sequence']][0:2800]

lncORF_sel.columns = ['Gene', 'tr_id1', 'Start', 
                    'Stop', 'Global_Rank_within_ORF_type', 
                    'Type_of_ORF', 'Amino acid sequence']

In [501]:
comb = pd.concat([uORF_sel, uoORF_sel, 
                  nORF_sel, 
                  #lncORF_sel,
                  doORF_sel, 
                  dORF_sel])

In [509]:
comb.shape[0]

8405

In [510]:
comb_df = comb.merge(metadata_pc_g25[['tr_id1', 'tr_id']], on='tr_id1', how='inner').drop_duplicates()

comb_df = comb_df[~comb_df['tr_id'].str.contains('PAR')]

print (comb_df.shape[0])

8404


In [511]:
lncORF_sel2 = lncORF_sel.merge(gtf_tr[['tr_id', 'tr_id1']], on='tr_id1', how='inner')

lncORF_sel.shape[0], lncORF_sel2.shape[0]

(2800, 2800)

In [512]:
total = pd.concat([comb_df, lncORF_sel2], axis=0)

total.shape[0]

11204

In [513]:
total.to_csv('tmp_res/11204_ncORFs.txt', sep='\t', index=False)

In [514]:
total[0:5]

,Gene,tr_id1,Start,Stop,Global_Rank_within_ORF_type,Type_of_ORF,Amino acid sequence,tr_id
0,SNX17,ENST00000233575,118,147,1,uorf,MASEAAGTR,ENST00000233575.6
1,SNX17,ENST00000233575,48,128,2342,uorf,MPRRLRPRTDVPPPSHIGSQGSQNGE,ENST00000233575.6
2,MRPL18,ENST00000367034,49,84,2,uorf,MAAPGERLSRP,ENST00000367034.4
3,MRPL18,ENST00000367034,244,306,88,nested,MKLSPQNSPTGTPGTWSFYL,ENST00000367034.4
4,CCNK,ENST00000389879,18,50,3,uorf,MAAVGKERRR,ENST00000389879.9


# GENOMIC COORDINATES

In [515]:
def prepare_global_coo(path_to_file_with_global_coo, colname):
    global_coo_g25 = pd.read_csv(path_to_file_with_global_coo, sep='\t')
    global_coo_g25_f = global_coo_g25[global_coo_g25['hit'] == True]
    global_coo_g25_f = global_coo_g25_f.sort_values(by=['seqnames', 'start'])
    
    global_coo_g25_f['global_coo'] = global_coo_g25_f['seqnames']+':'+global_coo_g25_f['start'].astype(str) +'-'+global_coo_g25_f['end'].astype(str)
    global_coo_g25_f = global_coo_g25_f[['group', 'group_name', 'strand', 'global_coo']].groupby(['group', 'group_name', 'strand']).agg('+'.join).reset_index()
    global_coo_g25_f.columns = ['group', 'tr_id', 'strand', colname]
    return global_coo_g25_f

In [517]:
coo = pd.read_csv('tmp_res/11204_ncORFs_global.txt', sep='\t')

coo[0:10]

,group,group_name,seqnames,start,end,width,strand,exon_id,exon_name,exon_rank,hit
1,1,ENST00000233575.6,chr2,27370639,27370668,30,+,64659,ENSE00000962997.3,1,True
2,1,ENST00000233575.6,chr2,27371269,27371268,0,+,64667,ENSE00003541824.1,2,False
3,1,ENST00000233575.6,chr2,27372623,27372622,0,+,64671,ENSE00003573225.1,3,False
4,1,ENST00000233575.6,chr2,27373247,27373246,0,+,64677,ENSE00003669336.1,4,False
5,1,ENST00000233575.6,chr2,27373861,27373860,0,+,64680,ENSE00003653420.1,5,False
6,1,ENST00000233575.6,chr2,27374085,27374084,0,+,64683,ENSE00003533996.1,6,False
7,1,ENST00000233575.6,chr2,27374346,27374345,0,+,64685,ENSE00003476021.1,7,False
8,1,ENST00000233575.6,chr2,27374689,27374688,0,+,64689,ENSE00003556896.1,8,False
9,1,ENST00000233575.6,chr2,27375061,27375060,0,+,64692,ENSE00003692961.1,9,False
10,1,ENST00000233575.6,chr2,27375506,27375505,0,+,64696,ENSE00003690576.1,10,False


In [520]:
all_ORFs_coo = prepare_global_coo(path_to_file_with_global_coo='tmp_res/11294_ncORFs_global.txt',
                                   colname = 'global_coo')

print (all_ORFs_coo.shape[0])

11204


In [522]:
total['group'] = list(range(1, total.shape[0]+1))

total.shape[0]

11204

In [523]:
total_global = total.merge(all_ORFs_coo, on=['tr_id', 'group'], how='inner')

total_global.shape[0]

11204

In [524]:
total_global[0:2]

,Gene,tr_id1,Start,Stop,Global_Rank_within_ORF_type,Type_of_ORF,Amino acid sequence,tr_id,group,strand,global_coo
0,SNX17,ENST00000233575,118,147,1,uorf,MASEAAGTR,ENST00000233575.6,1,+,chr2:27370639-27370668
1,SNX17,ENST00000233575,48,128,2342,uorf,MPRRLRPRTDVPPPSHIGSQGSQNGE,ENST00000233575.6,2,+,chr2:27370569-27370649


In [525]:
total_global.to_csv('tmp_res/11204_ncORFs_with_global_coo.txt', sep='\t', index=False)


In [526]:
total_global['Amino acid sequence'].nunique(), total_global['global_coo'].nunique()

(10997, 11072)

In [533]:
total_global

,Gene,tr_id1,Start,Stop,Global_Rank_within_ORF_type,Type_of_ORF,Amino acid sequence,tr_id,group,strand,global_coo
0,SNX17,ENST00000233575,118,147,1,uorf,MASEAAGTR,ENST00000233575.6,1,+,chr2:27370639-27370668
1,SNX17,ENST00000233575,48,128,2342,uorf,MPRRLRPRTDVPPPSHIGSQGSQNGE,ENST00000233575.6,2,+,chr2:27370569-27370649
2,MRPL18,ENST00000367034,49,84,2,uorf,MAAPGERLSRP,ENST00000367034.4,3,+,chr6:159790514-159790549
3,MRPL18,ENST00000367034,244,306,88,nested,MKLSPQNSPTGTPGTWSFYL,ENST00000367034.4,4,+,chr6:159791009-159791071
4,CCNK,ENST00000389879,18,50,3,uorf,MAAVGKERRR,ENST00000389879.9,5,+,chr14:99481426-99481458
...,...,...,...,...,...,...,...,...,...,...,...
11199,CTD-2341M24.1,ENST00000553668,157,219,2794,noncoding,MTFLLFAFGLASFIFLIHRG,ENST00000553668.2,11200,+,chr14:86070942-86070967+chr14:86127950-86127986
11200,CTC-351M12.1,ENST00000624833,206,286,2795,noncoding,MSIHLGTFPSSAFEDGCARQAVKDKH,ENST00000624833.1,11201,-,chr5:69189167-69189247
11201,RP11-1070N10.5,ENST00000555032,121,177,2798,noncoding,MDKLRAQWVMTCPESWRE,ENST00000555032.1,11202,+,chr14:95639953-95640009
11202,CTC-575D19.1,ENST00000522551,226,645,2799,noncoding,MVKSCGKDGFHIRVRLHSFHVIRINKMLSCAGADRLQTGMRGAFGK...,ENST00000522551.1,11203,+,chr5:168616577-168616996


# Group ncORFs based on genomic coordinates (e.g. [nORF,uORF], [tr_id1,tr_id2]) -> how many ncORFs unique by genomic coo and aa?

In [527]:
total_global = total.merge(all_ORFs_coo, on=['tr_id', 'group'], how='inner')

total_global['Start'] = total_global['Start'].astype(str)

total_global['Stop'] = total_global['Stop'].astype(str)

total_global['Global_Rank_within_ORF_type'] = total_global['Global_Rank_within_ORF_type'].astype(str)

total_global.shape[0]

11204

In [528]:
total_global['tr_id'] = total_global.groupby(['global_coo'])['tr_id'].transform(lambda x : ','.join(x))

total_global['Type_of_ORF'] = total_global.groupby(['global_coo'])['Type_of_ORF'].transform(lambda x : ','.join(x))

total_global['Start'] = total_global.groupby(['global_coo'])['Start'].transform(lambda x : ','.join(x))

total_global['Stop'] = total_global.groupby(['global_coo'])['Stop'].transform(lambda x : ','.join(x))

#total_global['Start'] = total_global.groupby(['global_coo'])['Start'].transform(lambda x : ','.join(x))

total_global['Global_Rank_within_ORF_type'] = total_global.groupby(['global_coo'])['Global_Rank_within_ORF_type'].transform(lambda x : ','.join(x))

In [531]:
total_global_agg = total_global[['tr_id', 'Gene', 'Start', 'Stop', 'Type_of_ORF',
                                'Global_Rank_within_ORF_type', 'global_coo', 'strand', 
                                'Amino acid sequence']].drop_duplicates()

total_global_agg.groupby(by='global_coo').count().sort_values(by='tr_id')

,tr_id,Gene,Start,Stop,Type_of_ORF,Global_Rank_within_ORF_type,strand,Amino acid sequence
global_coo,,,,,,,,
chr10:100185661-100185771,1,1,1,1,1,1,1,1
chr3:172711092-172711148,1,1,1,1,1,1,1,1
chr3:172750699-172750743,1,1,1,1,1,1,1,1
chr3:172750733-172750807,1,1,1,1,1,1,1,1
chr3:172750792-172750824,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
chr6:31666887-31666903+chr6:31667868-31667946,2,2,2,2,2,2,2,2
chr11:65423982-65424071,2,2,2,2,2,2,2,2
chr17:42130406-42130471,2,2,2,2,2,2,2,2


In [530]:
total_global_agg[total_global_agg['global_coo'] == 'chr14:20684620-20684682']

,tr_id,Gene,Start,Stop,Type_of_ORF,Global_Rank_within_ORF_type,global_coo,strand,Amino acid sequence
1534,"ENST00000336811.10,ENST00000555835.2,ENST00000...",ANG,"444,521,34","506,583,96","uorf,uorf,uorf","828,829,832",chr14:20684620-20684682,+,MQSRIQKNIFAFSTGSPSSY
1536,"ENST00000336811.10,ENST00000555835.2,ENST00000...",RNASE4,"444,521,34","506,583,96","uorf,uorf,uorf","828,829,832",chr14:20684620-20684682,+,MQSRIQKNIFAFSTGSPSSY
1540,"ENST00000336811.10,ENST00000555835.2,ENST00000...",RP11-903H12.5,"444,521,34","506,583,96","uorf,uorf,uorf","828,829,832",chr14:20684620-20684682,+,MQSRIQKNIFAFSTGSPSSY


In [339]:
total_global_agg

,tr_id,Gene,Start,Stop,Type_of_ORF,Global_Rank_within_ORF_type,global_coo,strand,Amino acid sequence
0,ENST00000233575.6,SNX17,118,147,uorf,1,chr2:27370639-27370668,+,MASEAAGTR
1,ENST00000233575.6,SNX17,48,128,uorf,2342,chr2:27370569-27370649,+,MPRRLRPRTDVPPPSHIGSQGSQNGE
2,ENST00000367034.4,MRPL18,49,84,uorf,2,chr6:159790514-159790549,+,MAAPGERLSRP
3,ENST00000389879.9,CCNK,18,50,uorf,3,chr14:99481426-99481458,+,MAAVGKERRR
4,ENST00000360150.8,ERGIC2,11,52,uorf,4,chr12:29371658-29371670+chr12:29381115-29381143,-,MAVGWDHNTSMTI
...,...,...,...,...,...,...,...,...,...
8295,ENST00000424112.2,HDAC11-AS1,32,64,noncoding,1992,chr3:13479990-13480022,-,MSCSCASQAG
8296,ENST00000416928.6,HAGLR,610,645,noncoding,1993,chr2:176177424-176177459,-,MIGYSSDLALP
8297,ENST00000427161.1,RP11-87N24.3,89,139,noncoding,1995,chr9:10631768-10631785+chr9:10632083-10632115,-,MVFFFFFFFFFFFFLC
8298,ENST00000609661.5,LINC00452,726,899,noncoding,1999,chr13:113919549-113919654+chr13:113920723-1139...,+,MPSTPSEFPAAALGPAQRAWAQHLETPKRQTGSCPGFVRSLPRLPS...


# Overlap with HLA-peptidomics data 

In [534]:
# only unique aa sequences 

unique_aa = list(total_global['Amino acid sequence'].unique())

print (len(unique_aa))

10997


In [535]:
total_global.shape[0]

11204

In [536]:
df = pd.read_csv('DATA/Novels_forAlla.txt', sep='\t')
df[0:4]

,Peptide,Gene_Names,BestPSM_PEP,ENST_ids,Protein_Accessions,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Parsed_Loci
0,AAAAPESF,CIZ1,0.010166,ENST00000476727,NonCoding_2_ENST00000476727_ENSG00000148337,9,128161251,128204383,-
1,AADGHHRGL,HSPBP1,0.001552,"ENST00000255631,ENST00000433386,ENST0000058569...","10_ENST00000255631_ENSG00000133265_F1_orf2,10_...",19,55262223,55280381,-
2,AADHGPDEY,POMK,0.001915,"ENST00000331373,ENST00000518991,ENST0000061442...","19_ENST00000331373_ENSG00000185900_F2_orf2,19_...",8,43093498,43131180,-
3,AAEPVDSSL,-,0.046727,-,ERV_1792085_ERVL-E-int::chr18:74213725-74215142_6,-,-,-,chr18:74213725-74215142


In [537]:
df['Protein_Accessions']

0             NonCoding_2_ENST00000476727_ENSG00000148337
1       10_ENST00000255631_ENSG00000133265_F1_orf2,10_...
2       19_ENST00000331373_ENSG00000185900_F2_orf2,19_...
3       ERV_1792085_ERVL-E-int::chr18:74213725-74215142_6
4       2_ENST00000538147_ENSG00000177990_F0_orf2,UTR5...
                              ...                        
1518          28_ENST00000503004_ENSG00000080822_F1_orf11
1519    ERV_1270162_MLT1K_MLT1K-int::chr15:29477473-29...
1520         ERV_2415898_MLT1M::chr21:37752264-37752370_1
1521    153_ENST00000335877_ENSG00000125818_F2_orf81,U...
1522          NonCoding_3_ENST00000325349_ENSG00000213235
Name: Protein_Accessions, Length: 1523, dtype: object

In [538]:
li = []

for i, aa in enumerate(unique_aa):
    for peptide in df['Peptide'].tolist():
        if aa.find(peptide) != -1:
            li.append([aa, peptide, aa.find(peptide)])
    if i % 100 == 0:
        print (i)
            

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900


In [547]:
outdf = pd.DataFrame(li, columns = ['Amino acid sequence', 'peptide', 'pos'])

In [549]:
found_ORFs = outdf.merge(total_global_agg, 
                         on='Amino acid sequence', how='inner').drop_duplicates()

In [551]:
found_ORFs.to_csv('send/peptides_found_in_ncORFs.txt', sep='\t', index=False)

In [553]:
found_ORFs.peptide.nunique(), found_ORFs['Amino acid sequence'].nunique(), found_ORFs['global_coo'].nunique()

(110, 94, 94)

In [557]:
found_ORFs.sort_values(by='Global_Rank_within_ORF_type', ascending=False)

,Amino acid sequence,peptide,pos,tr_id,Gene,Start,Stop,Type_of_ORF,Global_Rank_within_ORF_type,global_coo,strand
94,MEEMVMIMASVGFVWSSPGLMEVGVGGPVVGGMGLLQEDLISEFLF...,FLFQDFLRQA,43,ENST00000229390.7,SRSF9,393,566,nested,98,chr12:120464090-120464122+chr12:120465627-1204...,-
95,MEEMVMIMASVGFVWSSPGLMEVGVGGPVVGGMGLLQEDLISEFLF...,SEFLFQDF,41,ENST00000229390.7,SRSF9,393,566,nested,98,chr12:120464090-120464122+chr12:120465627-1204...,-
91,MGGGGVGPLCHELQSHHFQKGHHR,HELQSHHF,10,ENST00000320623.9,NQO1,614,688,nested,934,chr16:69718250-69718253+chr16:69718370-69718440,-
93,MDSWCSPSWRMGAFPMRKISLRPSEEPVMEKP,SEEPVMEKP,23,ENST00000394231.7,MIEN1,501,599,nested,93,chr17:39729563-39729605+chr17:39729685-39729740,-
9,MAGELQGTQAPSLRRSLG,GELQGTQAP,2,ENST00000368521.9,C1ORF43,90,146,uorf,919,chr1:154220461-154220517,-
...,...,...,...,...,...,...,...,...,...,...,...
38,MRRSASAARLAPLRHAC,SASAARLAPL,3,ENST00000269593.4,IGFBP4,45,98,uorf,1193,chr17:40443505-40443558,+
36,MELRNCVWSQPSLDAQEDAGGTVSEAWCQSCADPLLAMEQQAQRPS...,MEQQAQRPSP,37,ENST00000394697.6,ADCY7,69,248,uorf,1137,chr16:50266584-50266587+chr16:50287912-50288087,+
83,MSENVHFWDA,SENVHFWDA,1,ENST00000311177.6,HOXB9,78,110,ouorf,112,chr17:48626247-48626279,-
82,MWTELGGPEPEPKRDPEPEPEPAAECQHAALPGTPAA,AECQHAALP,23,ENST00000377698.3,ALDH1B1,72,185,ouorf,109,chr9:38392735-38392807+chr9:38395740-38395780,+
